# Download gridded radar data from thredds

In [1]:
import pandas as pd
import datetime
import numpy as np
import xarray as xr
import netCDF4
import matplotlib.pyplot as plt
import tqdm

### challenges: 
- takes alot of time, so the script must be able to restart from crash
    - store latest successfully stored datetime
    - create netcdf first, save iteratively
- takes alot of data, allocate before running the loop
    - store as int in multiplies of 100

In [2]:
start = '2021-12-01'
end = '2021-12-31'
time = pd.date_range(start, end, freq="5min").time

# days to iterate
days = []
for day in pd.date_range(start, end).tolist():
    days.append(day)

# Download daily raw files

In [3]:
# download metadata
link = 'https://thredds.met.no/thredds/dodsC/remotesensing/reflectivity-nordic/2021/12/yrwms-nordic.mos.pcappi-0-dbz.noclass-clfilter-novpr-clcorr-block.nordiclcc-1000.20211231.nc'
ds = netCDF4.Dataset(link)
lat_norway = ds['lat'][:].data
lon_norway = ds['lon'][:].data
block_norway = ds['block_percent'][:].data

# save
np.save('/media/erlend/a7db2311-330d-408b-b1d2-57343136083f/radar_winter_2021/lat_norway', lat_norway)
np.save('/media/erlend/a7db2311-330d-408b-b1d2-57343136083f/radar_winter_2021/lon_norway', lon_norway)
np.save('/media/erlend/a7db2311-330d-408b-b1d2-57343136083f/radar_winter_2021/block_norway', block_norway[0])
ds.close()

In [4]:
# Download day by day arrays
# For through days:
time_ind_current = 0
time_error = [] # retry these afterwards
days_iterate = days[time_ind_current:]
for date in tqdm.tqdm(days_iterate): 
    day = date.strftime('%d')
    month = date.strftime('%m')
    year = date.strftime('%Y')
    
    # setup link to threds: 
    try:
        link = 'https://thredds.met.no/thredds/dodsC/remotesensing/reflectivity-nordic/'+ year + '/' + month +'/yrwms-nordic.mos.pcappi-0-dbz.noclass-clfilter-novpr-clcorr-block.nordiclcc-1000.'+year+month+day+'.nc'
        ds = netCDF4.Dataset(link)

        # get times for this day
        times = ds['time'][:].data

        # download precipitation ammounts for all gridcells
        precipitation = ds.variables['lwe_precipitation_rate'][:, :, :].data.astype('float16')
        np.save('/media/erlend/a7db2311-330d-408b-b1d2-57343136083f/radar_winter_2021/prec_' +str(year)+str(month)+str(day), precipitation)
        np.save('/media/erlend/a7db2311-330d-408b-b1d2-57343136083f/radar_winter_2021/times_' +str(year)+str(month)+str(day), times)

    finally:
        ds.close()
        del precipitation
        del times
        del ds
        time_error.append(date) # in case we miss something, go back

  0%|                                                                                                                  | 0/31 [00:00<?, ?it/s]/tmp/ipykernel_3980736/572188711.py:20: RuntimeWarning: overflow encountered in cast
  precipitation = ds.variables['lwe_precipitation_rate'][:, :, :].data.astype('float16')
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 31/31 [1:48:56<00:00, 210.84s/it]


In [ ]:
# overflow values are discarded

# Read daily precipitation and write to netcdf

In [3]:
import pandas as pd
import datetime
import numpy as np
import xarray as xr
import netCDF4
import matplotlib.pyplot as plt
import tqdm

In [4]:
lat_norway = np.load('/media/erlend/a7db2311-330d-408b-b1d2-57343136083f/radar_winter_2021/lat_norway.npy')
lon_norway = np.load('/media/erlend/a7db2311-330d-408b-b1d2-57343136083f/radar_winter_2021/lon_norway.npy')
block_norway = np.load('/media/erlend/a7db2311-330d-408b-b1d2-57343136083f/radar_winter_2021/block_norway.npy')

In [5]:
# Create a new netCDF file
dataset = netCDF4.Dataset('/media/erlend/a7db2311-330d-408b-b1d2-57343136083f/radar_winter_2021/weather_data.nc', 'w', format='NETCDF4_CLASSIC')

# Define the dimensions
time_dim = dataset.createDimension('time', None)  # the unlimited dimension (None, can be appended to)
y_dim = dataset.createDimension('y', 2134)
x_dim = dataset.createDimension('x', 1694)

# Define the variables
times = dataset.createVariable('time', np.float64, ('time',))
times.units = 'minutes since 2022-01-01 00:00:00'
times.calendar = 'gregorian'

ys = dataset.createVariable('y', np.int32, ('y',))
xs = dataset.createVariable('x', np.int32, ('x',))
lats = dataset.createVariable('latitude', np.float32, ('y', 'x'))  # latitudes
lons = dataset.createVariable('longitude', np.float32, ('y', 'x'))  # longitudes
block = dataset.createVariable('block', np.float32, ('y', 'x'))  # longitudes
values = dataset.createVariable('value', np.int16, ('time', 'y', 'x'))  # the data variable

# Set the variable values
xs[:] = np.arange(1694)
ys[:] = np.arange(2134)

lats[:, :] = lat_norway
lons[:, :] = lon_norway
block[:, :] = block_norway[0]


In [6]:
# For through days:
i = 0
for date in tqdm.tqdm(days): 
    day = date.strftime('%d')
    month = date.strftime('%m')
    year = date.strftime('%Y')
    
    # get times for this day
    times_read = np.load('/media/erlend/a7db2311-330d-408b-b1d2-57343136083f/radar_winter_2021/times_' +str(year)+str(month)+str(day)+'.npy')
    precipitation = (np.load('/media/erlend/a7db2311-330d-408b-b1d2-57343136083f/radar_winter_2021/prec_' +str(year)+str(month)+str(day)+'.npy')*100).astype('int16')
    isnan = np.isnan(precipitation)
    precipitation[isnan] == -1
    times_read = np.array([datetime.datetime.utcfromtimestamp(i) for i in times_read])

    # write to NETCDF in chunks (speeds up later reading)
    times[i:i+len(times_read)] = netCDF4.date2num(times_read, units=times.units , calendar=times.calendar)
    values[i:i+len(times_read), :, :] = precipitation
    i += len(times_read)


    
    # # write to NETCDF
    # for j in range(len(times_read)):
    #     # Update the time variable
    #     times[i] = netCDF4.date2num(times_read[j], units=times.units , calendar=times.calendar)
    
    #     # Load your actual image data for this time step here
    #     values[i, :, :] = precipitation[j]
    #     i += 1
    
# Close the file
dataset.close()

  0%|                                          | 0/31 [00:00<?, ?it/s]/tmp/ipykernel_4006447/112035007.py:10: RuntimeWarning: overflow encountered in multiply
  precipitation = (np.load('/media/erlend/a7db2311-330d-408b-b1d2-57343136083f/radar_winter_2021/prec_' +str(year)+str(month)+str(day)+'.npy')*100).astype('int16')
/tmp/ipykernel_4006447/112035007.py:10: RuntimeWarning: invalid value encountered in cast
  precipitation = (np.load('/media/erlend/a7db2311-330d-408b-b1d2-57343136083f/radar_winter_2021/prec_' +str(year)+str(month)+str(day)+'.npy')*100).astype('int16')
100%|█████████████████████████████████| 31/31 [25:20<00:00, 49.05s/it]
